In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

tqdm.pandas()

In [2]:
data = pd.read_csv('../ai_medreview/data/data.csv')

In [3]:
data.head()

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better
0,2024-01-01 10:51:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,4.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN
1,2024-01-01 15:20:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,5.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN
2,2024-01-01 17:47:00,Very good,NaN,NaN,Earls-Court-Surgery,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,2.0,2,5.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-02 07:58:00,Good,PERSON was amazing kind and helpful The recept...,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,positive,0.937987,neutral,0.0,23.0,0,4.0,Reception Staff Interaction,NaN,mannual,upload,admiration,NaN
4,2024-01-02 10:44:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.0,1.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN


In [4]:
from ai_medreview.data import *

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the ch

In [5]:
raw_data = load_google_sheet()

2024-08-27 03:39:37.484 | INFO     | ai_medreview.utils:wrapper:16 - 🖥️    Started: 'load_google_sheet'
2024-08-27 03:39:40.591 | INFO     | ai_medreview.utils:wrapper:22 - ✅ Completed: 'load_google_sheet' ⚡️3.105744 sec


In [6]:
raw_data.head()

,submission_id,respondent-id,time,rating,free_text,do_better,pcn,surgery
0,mannual,upload,2024-01-01 10:51:00,Very good,Everyone kind and helpful,,Brompton-Health-PCN,Scarsdale-Medical-Centre
1,mannual,upload,2024-01-01 15:20:00,Very good,Nothing. It was all great,,Brompton-Health-PCN,Scarsdale-Medical-Centre
2,,,2024-01-01 17:47:25,Very good,Knowledgable doctors,All Likely,Brompton-Health-PCN,Earls-Court-Surgery
3,mannual,upload,2024-01-02 07:58:00,Good,"Denise was amazing, kind and helpful. The rece...",,Brompton-Health-PCN,Scarsdale-Medical-Centre
4,mannual,upload,2024-01-02 10:44:00,Very good,Appointment,,Brompton-Health-PCN,Scarsdale-Medical-Centre


In [7]:
raw_data.shape

(7850, 8)

In [8]:
data.shape

(7850, 19)

In [9]:
def get_person_names_with_transformers(text):
    # Check if the text is empty or not a string
    if not text or not isinstance(text, str):
        return []  # Return an empty list if the input is invalid or empty

    # Initialize a list to store person names
    person_names = []

    try:
        # Run the NER pipeline on the valid input text
        entities = ner_pipeline(text)

        # Iterate over detected entities
        for entity in entities:
            # Check if the entity is classified as a person
            if entity["entity_group"] == "PER":
                # Add the detected name to the list of person names
                person_names.append(entity["word"])
    except ValueError as e:
        # Log the error for debugging
        print(f"Error processing text: {text}")
        raise e

    return person_names

In [10]:

full_list = []

# Using tqdm to wrap around the iterable
for text in tqdm(raw_data['free_text'], desc="Processing texts"):
    person_list = get_person_names_with_transformers(text)
    personlist_len = len(person_list)
    if personlist_len != 0:
        full_list.append(person_list)
    else:
        full_list.append(np.nan)

Processing texts: 100%|█████████████████████████████████████████████████████████████████████████████| 7850/7850 [13:15<00:00,  9.87it/s]


In [11]:
full_list

[nan,
 nan,
 nan,
 ['Denise'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Emiliani'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Emiliani'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Like'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Clay'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Constantine'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Lula'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,


In [13]:
new = {
    'staff_names': full_list
}
new_df = pd.DataFrame(new)

In [14]:
new_df.head()

,staff_names
0,NaN
1,NaN
2,NaN
3,[Denise]
4,NaN


In [15]:
new_df.shape

(7850, 1)

In [20]:
merged = data.merge(new_df, left_index=True, right_index=True, how='left')

In [21]:
merged

,time,rating,free_text,do_better,surgery,pcn,sentiment_free_text,sentiment_score_free_text,sentiment_do_better,sentiment_score_do_better,free_text_len,do_better_len,rating_score,feedback_labels,improvement_labels,submission_id,respondent-id,emotion_free_text,emotion_do_better,staff_names
0,2024-01-01 10:51:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.000000,4.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN,NaN
1,2024-01-01 15:20:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.000000,5.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN,NaN
2,2024-01-01 17:47:00,Very good,NaN,NaN,Earls-Court-Surgery,Brompton-Health-PCN,neutral,0.000000,neutral,0.000000,2.0,2,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-02 07:58:00,Good,PERSON was amazing kind and helpful The recept...,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,positive,0.937987,neutral,0.000000,23.0,0,4.0,Reception Staff Interaction,NaN,mannual,upload,admiration,NaN,[Denise]
4,2024-01-02 10:44:00,Very good,NaN,NaN,Scarsdale-Medical-Centre,Brompton-Health-PCN,neutral,0.000000,neutral,0.000000,1.0,0,5.0,NaN,NaN,mannual,upload,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7845,2024-08-27 01:35:39,Very good,I struggle to use the surgery website I know i...,Improve the surgery website with information a...,Earls-Court-Surgery,Brompton-Health-PCN,negative,0.889186,neutral,0.705834,24.0,9,5.0,Surgery Website,Surgery Website,qeBWY7,P25QoB,disappointment,neutral,NaN
7846,2024-08-27 01:36:11,Very good,I would like to get a repeat prescription but ...,NaN,Earls-Court-Surgery,Brompton-Health-PCN,neutral,0.549259,neutral,0.000000,16.0,0,5.0,Prescriptions and Medication Management,NaN,lZAWK5,P25QoB,confusion,NaN,NaN
7847,2024-08-27 01:37:43,Very good,I ve been using this GP practice for years and...,NaN,Earls-Court-Surgery,Brompton-Health-PCN,positive,0.972750,neutral,0.000000,65.0,0,5.0,Staff Professionalism,NaN,XyGYQV,P25QoB,approval,NaN,NaN
7848,2024-08-27 01:38:01,Very good,I m so glad I switched to this GP practice Fro...,NaN,Earls-Court-Surgery,Brompton-Health-PCN,positive,0.970295,neutral,0.000000,90.0,0,5.0,Staff Professionalism,NaN,oQBdgM,P25QoB,approval,NaN,NaN


In [35]:
raw_data['full_text'] = raw_data['free_text'] + " " + raw_data['do_better']

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [33]:
raw_data

,submission_id,respondent-id,time,rating,free_text,do_better,pcn,surgery,full_text
0,mannual,upload,2024-01-01 10:51:00,Very good,Everyone kind and helpful,,Brompton-Health-PCN,Scarsdale-Medical-Centre,0 Everyone kind ...
1,mannual,upload,2024-01-01 15:20:00,Very good,Nothing. It was all great,,Brompton-Health-PCN,Scarsdale-Medical-Centre,0 Everyone kind ...
2,,,2024-01-01 17:47:25,Very good,Knowledgable doctors,All Likely,Brompton-Health-PCN,Earls-Court-Surgery,0 Everyone kind ...
3,mannual,upload,2024-01-02 07:58:00,Good,"Denise was amazing, kind and helpful. The rece...",,Brompton-Health-PCN,Scarsdale-Medical-Centre,0 Everyone kind ...
4,mannual,upload,2024-01-02 10:44:00,Very good,Appointment,,Brompton-Health-PCN,Scarsdale-Medical-Centre,0 Everyone kind ...
...,...,...,...,...,...,...,...,...,...
7845,qeBWY7,P25QoB,2024-08-27 01:35:39,Very good,"I struggle to use the surgery website, I know ...",Improve the surgery website with information a...,Brompton-Health-PCN,Earls-Court-Surgery,0 Everyone kind ...
7846,lZAWK5,P25QoB,2024-08-27 01:36:11,Very good,"I would like to get a repeat prescription, but...",,Brompton-Health-PCN,Earls-Court-Surgery,0 Everyone kind ...
7847,XyGYQV,P25QoB,2024-08-27 01:37:43,Very good,I've been using this GP practice for years and...,,Brompton-Health-PCN,Earls-Court-Surgery,0 Everyone kind ...
7848,oQBdgM,P25QoB,2024-08-27 01:38:01,Very good,I'm so glad I switched to this GP practice! Fr...,,Brompton-Health-PCN,Earls-Court-Surgery,0 Everyone kind ...


In [30]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7850 entries, 0 to 7849
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   submission_id  7850 non-null   object        
 1   respondent-id  7850 non-null   object        
 2   time           7850 non-null   datetime64[ns]
 3   rating         7850 non-null   object        
 4   free_text      7850 non-null   object        
 5   do_better      7850 non-null   object        
 6   pcn            7850 non-null   object        
 7   surgery        7850 non-null   object        
dtypes: datetime64[ns](1), object(7)
memory usage: 552.0+ KB


In [36]:
count_list = []

for index, row in raw_data.iterrows():
    free_text = row['free_text']
    do_better = row['do_better']
    full_text = f"{free_text} {do_better}"
    count_list.append(full_text)

In [37]:
count_list

['Everyone kind and helpful ',
 'Nothing. It was all great ',
 'Knowledgable doctors All Likely',
 'Denise was amazing, kind and helpful. The reception staff were cold and rather indifferent. A more welcoming and friendly approach would be appreciated. ',
 'Appointment ',
 'Earlier date ',
 'No issues with me ',
 'The booking time ',
 ' ',
 'Love my Practice ',
 'We have had Extremely likely service from the NHS regarding my diabetes. All of my needs have been met. Iâ€™m very happy. ',
 ' ',
 ' ',
 'Very helpful  No',
 ' ',
 ' ',
 ' ',
 'Always happy.  No',
 'The staff polite feel welcome and comfortable whit doctor  No',
 'I’ve been with the Surgery for years. Likely service, even in Covid. Many thanks. No',
 ' ',
 'Very nice and helpful No',
 'All people who works there \nVery  kind team work are Extremely likely  ',
 'They are  very kind ',
 ' ',
 'I always get an appointment when I need it and the staff and doctors take the time to listen to you.  Having a coffee machine so you can

In [38]:
len(count_list)

7850

In [41]:
full_list =[]

for text in tqdm(count_list, desc="Processing texts"):
    person_list = get_person_names_with_transformers(text)
    personlist_len = len(person_list)
    if personlist_len != 0:
        full_list.append(person_list)
    else:
        full_list.append(np.nan)
    

Processing texts: 100%|█████████████████████████████████████████████████████████████████████████████| 7850/7850 [19:02<00:00,  6.87it/s]


In [42]:
full_list


[nan,
 nan,
 nan,
 ['Denise'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Emiliani'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Emiliani'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Like'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Clay'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Constantine'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 ['Lula'],
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,


In [43]:
new = {
    'full_text': count_list,
    'names': full_list
}

new_df = pd.DataFrame(new)

In [44]:
new_df

,full_text,names
0,Everyone kind and helpful,NaN
1,Nothing. It was all great,NaN
2,Knowledgable doctors All Likely,NaN
3,"Denise was amazing, kind and helpful. The rece...",[Denise]
4,Appointment,NaN
...,...,...
7845,"I struggle to use the surgery website, I know ...",NaN
7846,"I would like to get a repeat prescription, but...",NaN
7847,I've been using this GP practice for years and...,NaN
7848,I'm so glad I switched to this GP practice! Fr...,NaN


In [48]:
count = 1
for text in new_df['names']:
    text = str(text)
    if text != 'nan':
        print(f"{count} {text}")
        count = count +1

1 ['Denise']
2 ['Emiliani']
3 ['Emiliani']
4 ['Like']
5 ['Clay']
6 ['Constantine']
7 ['Lula']
8 ['Emiliani']
9 ['Emiliani']
10 ['Ade', 'Ade', 'Mary', 'God']
11 ['Leo Scott - Francis']
12 ['Adib']
13 ['Adib']
14 ['Lula', 'Adib']
15 ['Leben']
16 ['Maria Naymark', 'God']
17 ['Brodie']
18 ['Karen Rydings']
19 ['Orr']
20 ['Will', 'Logan', 'Louise']
21 ['John West']
22 ['Steeden', 'Philip Ambulai']
23 ['Hussein']
24 ['Hussein', 'No']
25 ['Hussein']
26 ['Louise', 'Louise']
27 ['John']
28 ['Likely']
29 ['Darwin']
30 ['Burhan Adib', 'Mohiddin k']
31 ['Orr']
32 ['Adib']
33 ['Burhan Adib', 'Mohiddin k']
34 ['Barlow']
35 ['Emiliani']
36 ['Harriet Wright']
37 ['Charlotte Moon', 'Olivia Orr', 'God']
38 ['Tim Rees']
39 ['Louise']
40 ['Orr', 'Stan', 'Chelsea']
41 ['S']
42 ['Savannah Eduvie', 'Cloyette Callender', 'God']
43 ['Logan', 'Orr', 'Amanda Stockton']
44 ['Adip']
45 ['A', 'E', 'Alex']
46 ['Adib', 'Adib']
47 ['Adib']
48 ['Adib']
49 ['Adib', 'Adib']
50 ['Christine', 'Joy Lula']
51 ['Emiliani Orie

In [49]:
import ipywidgets as widgets
from IPython.display import display

# Create a simple text widget
text_widget = widgets.Text(
    value='Hello, World!',
    placeholder='Type something',
    description='Text:',
)

# Display the text widget
display(text_widget)

# Create an interactive slider widget
slider_widget = widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Slider:',
    continuous_update=False
)

# Display the slider widget
display(slider_widget)

# Create a button widget
button_widget = widgets.Button(
    description='Click Me!',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'  # (FontAwesome names without the `fa-` prefix)
)

# Define a callback function to handle button click
def on_button_click(b):
    print(f"Button clicked with slider value: {slider_widget.value}")

# Attach the callback function to the button
button_widget.on_click(on_button_click)

# Display the button widget
display(button_widget)

# Create a checkbox widget
checkbox_widget = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False
)

# Display the checkbox widget
display(checkbox_widget)

Text(value='Hello, World!', description='Text:', placeholder='Type something')

IntSlider(value=7, continuous_update=False, description='Slider:', max=10)

Button(description='Click Me!', icon='check', style=ButtonStyle(), tooltip='Click me')

Checkbox(value=False, description='Check me')